In [259]:
# %load TestHarness
debugging = False
#debugging = True

debugging2 = False

logging = True


def dbg(f, *args):
    if debugging:
        print(('  DBG:' + f).format(*args))

def dbg2(f, *args):
    if debugging2:
        print(('  DBG2:' + f).format(*args))
        
def log(f, *args):
    if logging:
        print((f).format(*args))
        
def log_error(f, *args):
    if logging:
        print(('*** ERROR:' + f).format(*args))       
        
def class_name(instance):
    return type(instance).__name__
     

In [260]:
# Test Harness
#------------------------------------------------------------------------------   
import time
from datetime import timedelta

#------------------------------------------------------------------------------
class TestCase(object):
    def __init__(self, name, method, inputs, expected, catchExceptions=False):
        self.name = name
        self.method = method
        self.inputs = inputs
        self.expected = expected
        self.catchExceptions = catchExceptions
        
    def run(self):
        if self.catchExceptions:
            try:
                return self.method(*self.inputs)
            except Exception as x:
                return x
        else:
                return self.method(*self.inputs)

#------------------------------------------------------------------------------
class TestSet(object):
    def __init__(self, cases):
        self.cases = cases
    
    def run_tests(self, repeat=1):
        count = 0
        errors = 0
        total_time = 0
        for case in self.cases:
            count += 1
            start_time = time.time()
            for iteration in range(repeat):
                dbg2("*** Running '{0}' iteration {1}", case.name, iteration+1)
                result = case.run()
            elapsed_time = time.time() - start_time
            total_time += elapsed_time
            if callable(case.expected):
                if not case.expected(result):
                    errors += 1
                    logError("Test {0} failed. Returned {1}", case.name, result)
            elif result != case.expected:
                errors += 1
                logError('Test {0} failed. Returned "{1}", expected "{2}"', case.name, result, case.expected)
        if errors:
            logError("Tests passed: {0}; Failures: {1}", count-errors, errors)
        else:
            log("All {0} tests passed.", count)
        log("Elapsed test time: {0}", timedelta(seconds=total_time))


In [261]:
xStack = []
def push(v):
    global xStack
    xStack.append(v)
def pop():
    global xStack
    return xStack.pop()

In [262]:
def digit_count_slow(n, d):
    ''' Get the number of times digit "d" appears in 0..n inclusive. '''
    def single_count(i, d):
        assert i >= 0
        assert d >=0 and d <= 9
        if i == 0: 
            return 0 if d > 0 else 1
        c = 0
        while i > 0:
            c += i % 10 == d
            i //= 10
        return c
    
    c = 0
    for i in range(n+1):
        c += single_count(i, d)
    return c
        

In [263]:
def digit_count_str(n, d=0):
    ''' Get the number of times digit "d" appears in 0..n inclusive. '''
    s = []
    d_char = str(d)
    for i in range(n+1): s += str(i)
    return sum([ch == d_char for ch in s])

In [264]:
def power10(n):
    ''' get the scale (1, 10, 100, 1000 ...) and the magnitude (power of 10) 
        Note 0 is give and scale of 1.
    '''
    assert n >= 0
    scale = 1
    mag = -1
    while scale <= n: 
        scale *= 10
        mag += 1
    if scale > 1: scale = scale // 10
    return scale, mag

In [313]:
def zdcount(n, digit, scale=-1, mag=0, zeros=False):
    ''' Count the number of digits in the range 0..n. 
    
        Including zero is significantly more challenging since
        leading zeros are only significant in 'interior' cases.
        
        Thus zd(3003, 0) is zd(0..3000) + zd(001,002,003)
        zd(4211) is 4*zd(000..999) + zd(000..211)
        
        The critical insight for this problem is that for any number
        xrrrr, there are two different evaluations of rrrr. For the
        first 9999, the leading zeros are not counted, but from 10000
        to xrrrr, we must count the leading zeros.
        
    '''
    if scale < 0: scale, mag = power10(n)
    if n < 10:
        if digit: return 1 if digit <= n else 0
        # The single digit case is tricky, since leading zeros are
        # are implied when a mag>0.
        if scale <= 1 or not zeros: return 1

    dig = n // scale
    rrr = n %  scale
    r99 = scale-1
    nmag = mag - 1
    nscale = scale // 10

    # Number of times the most significant digit is found.
    cnt = 0
    zzz = digit == 0
    
    if zeros or digit:
        if dig > digit:
            cnt = scale
        elif dig == digit:
            cnt = rrr + 1 # Account for this lone digit.
        if mag:
            cnt += zdcount(r99, digit, nscale, nmag, zzz) * dig               
    else:
        # First count the values up to 999, without the leading zeros:
        cnt = zdcount(r99, 0, nscale, nmag, zeros=False)
        if dig: 
            cnt += zdcount(r99, 0, nscale, nmag, zzz) * (dig-1) 
            
    if not zeros or (mag and (digit or zeros)):
        cnt += zdcount(rrr, digit, nscale, nmag, zzz)     
    return cnt

In [314]:
zdcount(1287330, 0)

734164

In [315]:
reff = zerod_count_chatty_nasty
uutf = zdcount
trials = [(reff(z), uutf(z, 0)) for z in range(0,10000)]
all([t[0] == t[1] for t in trials])
#trials

True

In [316]:
reff = digit_count_str
uutf = zdcount
trials = [(reff(z, d), uutf(z, d)) for z in range(0,1000) for d in range(0,10)]
all([t[0] == t[1] for t in trials])

True

In [317]:
[zdcount(n, 0) for n in range(9)]

[1, 1, 1, 1, 1, 1, 1, 1, 1]

In [98]:
def zerod_count_chatty_nasty(n, scale=0, mag=0):
    ''' Count the number of zero digits in the range 0..n
        This is distinctly different from d_count as leading zeros are
        always significant.
        
        Thus zd(3003) is zd(0..3000) + zd(001,002,003)
        zd(4211) is 4*zd(000..999) + zd(000..211)
        
        The critical insight for this problem is that for any number
        xrrrr, there are two different evaluations of rrrr. For the
        first 9999, the leading zeros do not exist, but from 10000
        to xrrrr, we must count the leading zeros.
        
    '''
    
    def zcount(n, scale=0, mag=0):
        if scale == 0: scale, mag = power10(n)
        X = '--'*(4-mag)
        dbg('{0} n={1} zzENTER scale={2}, mag={3}', X, str(n).zfill(mag+1), scale, mag)

        if n < 10 and scale==1:
            cnt = 1
            dbg('{0} n={1} (d)RETURN count={2}', X, str(n).zfill(mag+1), cnt)
            return cnt

        dig = n // scale
        rrr = n %  scale
        r99 = scale-1
        nmag = mag - 1
        nscale = scale // 10

        if dig == 0:
            cnt = rrr + 1 # Account for this lone zero.
            if mag: cnt += zcount(rrr, nscale, nmag)
        else:
            cnt = scale
            if mag:
                cnt += zcount(r99, nscale, nmag) * dig
                cnt += zcount(rrr, nscale, nmag)
        dbg('{0} n={1} zzRETURN count={2}', X, str(n).zfill(mag+1), cnt)            
        return cnt
    
    if scale == 0: scale, mag = power10(n)
    X = '--'*(4-mag)
    dbg('{0} n={1} zdENTER scale={2}, mag={3}', X, str(n).zfill(mag+1), scale, mag)        
    if n < 10:
        cnt = 1
        #dbg('n={0} zdRETURN count={1}', n, cnt)
        return cnt
    dig = n // scale
    rrr = n % scale
    nscale = scale // 10
    nmag = mag-1
    # First count the values up to 999
    cnt = zerod_count(scale-1, nscale, nmag)
    dbg('{0} n={1}   UNFILLED count={2}', X, str(scale-1).zfill(mag), cnt)
    if dig > 1:
        c = zcount(scale-1, nscale, nmag) * (dig-1)
        dbg('{0} n={1} ZEROFILLED count={2}', X, str(scale-1).zfill(mag), c)
        cnt += c
    #f rem > 0 or dig == 1:
    c = zcount(rrr, nscale, nmag)
    dbg('{0} n={1} REMAINDER count={2}', X, str(rrr).zfill(mag), c)
    cnt += c
    dbg('{0} n={1} count={2}', X, n, cnt)
    return cnt
